# GeoEnrichment

GeoEnrichment provides the ability to get facts about a location or area. Using GeoEnrichment, you can get information about the people and places in a specific area or within a certain distance or drive time from a location. It enables you to query and use information from a large collection of data sets including population, income, housing, consumer behavior, and the natural environment.

This module enables you to answer questions about locations that you can't answer with maps alone. For example: What kind of people live here? What do people like to do in this area? What are their habits and lifestyles?


The `enrich()` method to can be used retrieve demographics and other relevant characteristics associated with the area surrounding the requested places. You can also use the `arcgis.geoenrichment` module to obtain additional geographic context (for example, the ZIP Code of a location) and geographic boundaries (for example, the geometry for a drive-time service area).

Site analysis is a popular application of this type of data enrichment. For example, GeoEnrichment can be leveraged to study the population that would be affected by the development of a new community center within their neighborhood. With the `enrich()` method, the proposed site can be submitted, and the demographics and other relevant characteristics associated with the area around the site will be returned.

The following topics are covered in this guide
 - [Getting started](#Getting-Started)
 - [GeoEnrichment coverage](#GeoEnrichment-coverage)
   - [Filtering countries by properties](#Filtering-countries-by-properties)
   - [Discovering information for a country](#Discovering-information-for-a-country)
   - [Data collections and analysis variables](#Data-collections-and-analysis-variables)
   - [Available reports](#Available-reports)
 - [Finding named statistical areas](#Finding-named-statistical-areas)
   - [Searching for named areas within a country](#Searching-for-named-areas-within-a-country)
     - [Filtering named areas by geography level](#Filtering-named-areas-by-geography-level)
 - [Working with study areas](#Working-with-study-Areas)
   - [Accepted forms of study areas](#Accepted-forms-of-study-areas)
 - [Creating Reports](#Creating-Reports)
 - [Enriching study areas](#Enriching-study-areas)
   - [Enriching an existing feature layer](#Enriching-an-existing-feature-layer)
   - [Example: Enriching a named statistical area](#Example:-Enriching-a-named-statistical-area)
   - [Example: Enrich all counties in a state](#Example:-Enrich-all-counties-in-a-state)
   - [Example: Usage comparison levels](#Example:-Using-comparison-levels)
   - [Example: Enriching a street address](#Example:-Enriching-street-address)
   - [Example: Buffering locations using non-overlapping disks](#Example:-Buffering-locations-using-non-overlapping-disks)
   - [Example: Using drive times as study areas](#Example:-Using-drive-times-as-study-areas)
     - [Visualize results on a map](#Visualize-results-on-a-map)
 - [Saving GeoEnrichment results](#Saving-GeoEnrichment-Results)

## Getting Started
A user must be logged on to a GIS in order to use GeoEnrichment. The geoenrichment functionality is available in the `arcgis.geoenrichment` module.

In [1]:
from arcgis.gis import GIS
from arcgis.geoenrichment import *

gis = GIS('https://www.arcgis.com', 'arcgis_python', 'P@ssword123')

## GeoEnrichment coverage

The `get_countries()` method can be used to query the countries for which there is GeoEnrichment data. You get back a list of `Country` object. You can query the properties of each `Country` object.

In [6]:
countries = get_countries()
print("Number of countries for which GeoEnrichment data is available: " + str(len(countries)))

#print a few countries for a sample
countries[0:10]

Number of countries for which GeoEnrichment data is available: 137


[<Country name:Albania>,
 <Country name:Algeria>,
 <Country name:Andorra>,
 <Country name:Angola>,
 <Country name:Argentina>,
 <Country name:Armenia>,
 <Country name:Aruba>,
 <Country name:Australia>,
 <Country name:Austria>,
 <Country name:Azerbaijan>]

### Filtering countries by properties
You can filter out the `Country` objects based on one or more of their properties. For instance, the snippet below gets the countries in Australian continent.

In [5]:
[c.properties.name for c in countries if c.properties.continent == 'Australia']

['Australia',
 'French Polynesia',
 'Indonesia',
 'Malaysia',
 'New Caledonia',
 'New Zealand',
 'Philippines']

## Discovering information for a country
The `Country` class can be used to discover the data collections, sub-geographies and available reports for a country. When working with a particular country, you will find it convenient to get a reference to it using the `Country.get()` method. This method accepts the country name or its [2 letter abbreviation](http://www.nationsonline.org/oneworld/country_code_list.htm) or [ISO3 code](https://unstats.un.org/unsd/tradekb/knowledgebase/country-code) and returns an instance of that country.

In [10]:
usa = Country.get('US')
type(usa)

arcgis.geoenrichment.enrichment.Country

Commonly used properties for the country are accessible using `Country.properties`.

In [12]:
usa.properties.name

'United States'

In [13]:
usa.properties.datasets

['USA_ESRI_2017', 'USA_ACS_2017', 'USA_ASR_2017', 'USA_RMP_2017', 'Landscape']

### Data collections and analysis variables
GeoEnrichment uses the concept of a data collection to define the data attributes returned by the enrichment service. A data collection is a preassembled list of attributes that will be used to enrich the input features. Collection attributes can describe various types of information, such as demographic characteristics and geographic context of the locations or areas submitted as input features.

The `data_collections` property of a `Country` object lists its available data collections and analysis variables under each data collection as a [Pandas](https://pandas.pydata.org/) dataframe. 

In [14]:
df = usa.data_collections

# print a few rows of the DataFrame
df.head()

,analysisVariable,alias,fieldCategory,vintage
dataCollectionID,,,,
1yearincrements,1yearincrements.AGE0_CY,2017 Population Age <1,2017 Age: 1 Year Increments (Esri),2017
1yearincrements,1yearincrements.AGE1_CY,2017 Population Age 1,2017 Age: 1 Year Increments (Esri),2017
1yearincrements,1yearincrements.AGE2_CY,2017 Population Age 2,2017 Age: 1 Year Increments (Esri),2017
1yearincrements,1yearincrements.AGE3_CY,2017 Population Age 3,2017 Age: 1 Year Increments (Esri),2017
1yearincrements,1yearincrements.AGE4_CY,2017 Population Age 4,2017 Age: 1 Year Increments (Esri),2017


In [15]:
# call the shape property to get the total number of rows and columns
df.shape

(14609, 4)

Each data collection and analysis variable has a unique ID. When calling the `enrich()` method (explained later in this guide) these analysis variables can be passed in the `data_collections` and `analysis_variables` parameters.

You can filter the `data_collections` and query the collections `analysis_variables` using Pandas expressions.

In [16]:
# get all the unique data collections available
df.index.unique()

Index(['1yearincrements', '5yearincrements', 'ACS_Housing_Summary_rep',
       'ACS_Population_Summary_rep', 'Age', 'Age_50_Profile_rep',
       'Age_by_Sex_Profile_rep', 'Age_by_Sex_by_Race_Profile_rep', 'AtRisk',
       'AutomobilesAutomotiveProducts',
       ...
       'transportation', 'travelMPI', 'unitsinstructure',
       'urbanizationgroupsNEW', 'vacant', 'vehiclesavailable', 'veterans',
       'women', 'yearbuilt', 'yearmovedin'],
      dtype='object', name='dataCollectionID', length=141)

The snippet below shows how you can query the `Age` data collection and get all the unique `analysisVariable`s under that collection

In [17]:
df.loc['Age']['analysisVariable'].unique()

array(['Age.MALE0', 'Age.MALE5', 'Age.MALE10', 'Age.MALE15', 'Age.MALE20',
       'Age.MALE25', 'Age.MALE30', 'Age.MALE35', 'Age.MALE40',
       'Age.MALE45', 'Age.MALE50', 'Age.MALE55', 'Age.MALE60',
       'Age.MALE65', 'Age.MALE70', 'Age.MALE75', 'Age.MALE80',
       'Age.MALE85', 'Age.FEM0', 'Age.FEM5', 'Age.FEM10', 'Age.FEM15',
       'Age.FEM20', 'Age.FEM25', 'Age.FEM30', 'Age.FEM35', 'Age.FEM40',
       'Age.FEM45', 'Age.FEM50', 'Age.FEM55', 'Age.FEM60', 'Age.FEM65',
       'Age.FEM70', 'Age.FEM75', 'Age.FEM80', 'Age.FEM85'], dtype=object)

In [18]:
# view a sample of the `Age` data collection
df.loc['Age'].head()

,analysisVariable,alias,fieldCategory,vintage
dataCollectionID,,,,
Age,Age.MALE0,2017 Males Age 0-4,2017 Age: 5 Year Increments (Esri),2017
Age,Age.MALE5,2017 Males Age 5-9,2017 Age: 5 Year Increments (Esri),2017
Age,Age.MALE10,2017 Males Age 10-14,2017 Age: 5 Year Increments (Esri),2017
Age,Age.MALE15,2017 Males Age 15-19,2017 Age: 5 Year Increments (Esri),2017
Age,Age.MALE20,2017 Males Age 20-24,2017 Age: 5 Year Increments (Esri),2017


### Available reports
GeoEnrichment also enables you to create many types of high quality reports for a variety of use cases describing the input area.

The `reports` property of a `Country` object lists its available reports as a Pandas DataFrame. The report `id` you see below is used as an input in the `create_report()` method to create reports.

In [20]:
# print a sample of the reports available for USA
usa.reports.head(10)

,id,title,categories,formats
0,census2010_profile,2010 Census Profile,[Demographics],"[pdf, xlsx]"
1,acs_housing,ACS Housing Summary,[Demographics],"[pdf, xlsx]"
2,acs_population,ACS Population Summary,[Demographics],"[pdf, xlsx]"
3,55plus,Age 50+ Profile,[Demographics],"[pdf, xlsx]"
4,agesexrace,Age by Sex by Race Profile,[Demographics],"[pdf, xlsx]"
5,agesex,Age by Sex Profile,[Demographics],"[pdf, xlsx]"
6,cex_auto,Automotive Aftermarket Expenditures,[Consumer Spending],"[pdf, xlsx]"
7,business_loc,Business Locator,[Business],"[pdf, xlsx]"
8,business_summary,Business Summary,[Business],"[pdf, xlsx]"
9,community_profile,Community Profile,[Demographics],"[pdf, xlsx]"


In [21]:
# total number of reports available
usa.reports.shape

(49, 4)

## Finding named statistical areas

Each country has several named statistical areas in a hierarchy of geography levels (such as states, counties, zip codes, etc).

The `subgeographies` property of a country can be used to discover these standard geographic/statistical areas within that country. 

This information is available through a heirarchy of dynamic properties (like states, counties, tracts, zip-codes,...). Each such dynamic property reflects the geographical levels within that country, with subgeographies grouped logically under the higher levels of geographies. The properties are dictionaries containing the names of the standard geographic places and their values are instances of `NamedArea` class. The `NamedArea` objects can be used as **study areas** in the `enrich()` method.

<blockquote><b>Note:</b> Setting the `IPCompleter.greedy=True` configuration option in Jupyter notebook enables you to dynamically discover the various levels of subgeographies using intellisense, as in the example below:</blockquote>

In [22]:
%config IPCompleter.greedy=True

In [23]:
usa.subgeographies.states['California'].counties['San_Bernardino_County']

<NamedArea name:"San Bernardino County" area_id="06071", level="US.Counties", country="United States">

In [11]:
usa.subgeographies.states['California'].counties['San_Bernardino_County'].tracts['060710001.03']

<NamedArea name:"060710001.03" area_id="06071000103", level="US.Tracts", country="United States">

In [12]:
usa.subgeographies.states['California'].zip5['92373']

<NamedArea name:"Redlands" area_id="92373", level="US.ZIP5", country="United States">

The named areas can also be drawn on a map, as they include a `geometry` property.

In [24]:
m = gis.map('Redlands, CA', zoomlevel=11)
m

![img: Draw a subgeography](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_ge_subgeography_01.png)

In [25]:
m.draw(usa.subgeographies.states['California'].zip5['92373'].geometry)

The subgeography levels and their hierarchies are different for each country, and can also be different for the different datasets (which can be set using the `Country.dataset` property).

In [26]:
india = Country.get('India')

Print the available datasets.

In [27]:
india.properties.datasets

['IND_MBR_2016', 'IND_Indicus']

View the current dataset of the country.

In [28]:
india.dataset

'IND_MBR_2016'

Inspect the various subgeographies.

In [29]:
india.subgeographies.states['Uttar_Pradesh'].districts['Baghpat'].subdistricts['Baraut']

<NamedArea name:"Baraut" area_id="09080001", level="IN.Subdistricts", country="India">

Change the dataset used for the country. Notice the difference in available subgeographies.

In [21]:
india.dataset = 'IND_Indicus'
india.dataset

'IND_Indicus'

In [24]:
india.subgeographies.state['Uttar_Pradesh'].district['Baghpat']

<NamedArea name:"Baghpat" area_id="IND009139", level="IN.District", country="India">

### Searching for named areas within a country

In [31]:
riversides_in_usa = usa.search('Riverside')
print("number of riversides in the US: " + str(len(riversides_in_usa)))

# list a few of them
riversides_in_usa[:10]

number of riversides in the US: 83


[<NamedArea name:"Riverside" area_id="147435", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147436", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147437", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147438", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147439", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147440", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147441", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147442", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147443", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147444", level="Cities", country="United States">]

For instance, you can make a map of all the riversides in the US

In [32]:
usamap = gis.map('United States', zoomlevel=4)
usamap

![img: riversides in the us](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_ge_named_areas_02.png)

In [33]:
for riverside in riversides_in_usa:
    usamap.draw(riverside.geometry)

#### Filtering named areas by geography level

In [30]:
[level['id'] for level in usa.levels]

['US.WholeUSA',
 'US.States',
 'US.DMA',
 'US.CD',
 'US.CBSA',
 'US.Counties',
 'US.CSD',
 'US.ZIP5',
 'US.Places',
 'US.Tracts',
 'US.BlockGroups']

In [29]:
usa.search(query='Riverside', layers=['US.Counties'])

[<NamedArea name:"Riverside County" area_id="06065", level="US.Counties", country="United States">]

## Working with study Areas
GeoEnrichment uses the concept of a study area to define the location of the point or area that you want to enrich with additional information or create reports about.

### Accepted forms of study areas

- **Street address locations** - Locations can be passed as strings of input street addresses, points of interest or place names.
    + **Example:** `"380 New York St, Redlands, CA"`

- **Multiple field input addresses** - Locations described as multiple field input addresses, using dictionaries.
    + **Example:** 
        {"Address" : "380 New York Street",
        "City" : "Redlands",
        "Region" : "CA",
        "Postal" : 92373}    
 
- **Point and line geometries** - Point and line locations, using `arcgis.geometry` instances.
    + **Example Point Location: ** 
    
    `arcgis.geometry.Geometry({"x":-122.435,"y":37.785})`

- **Buffered study areas** - `BufferStudyArea` instances to change the ring buffer size or create drive-time service areas around points specified using one of the above methods. BufferStudyArea allows you to buffer point and street address study areas. They can be created using the following parameters:
        * area: the point geometry or street address (string) study area to be buffered
        * radii: list of distances by which to buffer the study area, eg. [1, 2, 3]
        * units: distance unit, eg. Miles, Kilometers, Minutes (when using drive times/travel_mode)
        * overlap: boolean, uses overlapping rings/network service areas when True, or non-overlapping disks when False
        * travel_mode: None or string, one of the supported travel modes when using network service areas
    + **Example Buffered Location: ** 
    
    `pt = arcgis.geometry.Geometry({"x":-122.435,"y":37.785})
    buffered_area = BufferStudyArea(area=pt, radii=[1,2,3], units="Miles", overlap=False)` 

- **Network service areas** - `BufferStudyArea` also allows you to define drive time service areas around points as well as other advanced service areas such as walking and trucking.
    + **Example: **
    
    `pt = arcgis.geometry.Geometry({"x":-122.435,"y":37.785})
    buffered_area = BufferStudyArea(area=pt, radii=[1,2,3], units="Minutes", travel_mode="Driving")` 

- **Named statistical areas** - In all previous examples of different study area types, locations were defined as either points or polygons. Study area locations can also be passed as one or many named statistical areas. This form of study area lets you define an area as a standard geographic statistical feature, such as a census or postal area, for example, to obtain enrichment information for a U.S. state, county, or ZIP Code or a Canadian province or postal code. When the NamedArea instances should be combined together (union), a list of such NamedArea instances should constitute a study area in the list of requested study areas.
    + **Example:** 
    
    `usa.subgeographies.states['California'].zip5['92373']`
   
- **Polygon geometries** - Locations can given as polygon geometries.
    + **Example Polygon geometry: ** 
    
    `arcgis.geometry.Geometry({"rings":[[[-117.185412,34.063170],[-122.81,37.81],[-117.200570,34.057196],[-117.185412,34.063170]]],"spatialReference":{"wkid":4326}})`


## Creating Reports
The `create_report` method allows you to create many types of high quality reports for a variety of use cases describing the input area. If a point is used as a study area, the service will create a `1` mile ring buffer around the point to collect and append enrichment data. Optionally, you can create a buffer ring or drive time service area around points of interest to generate PDF or Excel reports containing relevant information for the area on demographics, consumer spending, tapestry market, etc.

To find the list of reports available, refer to the section [Available reports](#Available-reports) earlier in this page.

In [28]:
report = create_report(study_areas=["380 New York Street, Redlands, CA"],
                     report="tapestry_profileNEW",
                     export_format="PDF", 
                     out_folder=r"c:\xc", out_name="esri_tapestry_profile.pdf")
report

'c:\\xc\\esri_tapestry_profile.pdf'

You can find a sample PDF report [here](http://help.arcgis.com/en/geoenrichment/rest-report-samples/ex2.pdf)

## Enriching study areas

The `enrich()` method returns a `SpatialDataFrame` that can either be used for mapping via the `gis.content.import_data()` method or for further analysis. If `return_geometry` is set to `False`, a Pandas DataFrame is returned instead.

### Enriching an existing feature layer
The examples to be shown below return you a new `SpatialDataFrame` that you can save as a new `Item` in your GIS, use for analysis or visualize on a map. However, if you would like to enrich an existing `FeatureLayer` by adding new data, then use [the `enrich_layer()` method](http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.enrich_data.html#enrich-layer) from the `features` module.

### Example: Enriching a named statistical area
Enriching zip code 92373 in California using the 'Age' data collection:

In [38]:
redlands = usa.subgeographies.states['California'].zip5['92373']

In [39]:
enrich(study_areas=[redlands], data_collections=['Age'] )

,FEM0,FEM10,FEM15,FEM20,FEM25,FEM30,FEM35,FEM40,FEM45,FEM5,...,MALE75,MALE80,MALE85,OBJECTID,StdGeographyID,StdGeographyLevel,StdGeographyName,aggregationMethod,sourceCountry,SHAPE
0,871,931,962,1123,1147,1206,1126,1041,1098,891,...,477,317,362,1,92373,US.ZIP5,Redlands,Query:US.ZIP5,US,"{'rings': [[[-117.21461999975689, 34.065140000..."


### Example: Enrich all counties in a state

In [40]:
ca_counties = usa.subgeographies.states['California'].counties

In [41]:
counties_df = enrich(study_areas=ca_counties, data_collections=['Age'])
counties_df.head(10)

,FEM0,FEM10,FEM15,FEM20,FEM25,FEM30,FEM35,FEM40,FEM45,FEM5,...,MALE75,MALE80,MALE85,OBJECTID,StdGeographyID,StdGeographyLevel,StdGeographyName,aggregationMethod,sourceCountry,SHAPE
0,47323,50068,50590,57125,60205,61600,59673,56910,56069,49093,...,16318,10422,10537,1,06001,US.Counties,Alameda County,Query:US.Counties,US,"{'rings': [[[-122.27167899966491, 37.904723999..."
1,24,33,29,20,24,24,23,25,34,33,...,17,10,5,2,06003,US.Counties,Alpine County,Query:US.Counties,US,"{'rings': [[[-119.90431499971437, 38.933323999..."
2,706,871,867,725,747,790,809,821,1019,769,...,789,510,495,3,06005,US.Counties,Amador County,Query:US.Counties,US,"{'rings': [[[-120.07763899938011, 38.708886999..."
3,6065,5940,8112,10888,7978,6721,5809,5610,6093,5981,...,3095,2107,2277,4,06007,US.Counties,Butte County,Query:US.Counties,US,"{'rings': [[[-121.40462100065271, 40.146641000..."
4,890,1117,1111,951,935,976,956,1036,1333,987,...,1011,572,447,5,06009,US.Counties,Calaveras County,Query:US.Counties,US,"{'rings': [[[-120.07246099958044, 38.509088000..."
5,851,812,687,643,762,731,665,616,638,835,...,241,150,146,6,06011,US.Counties,Colusa County,Query:US.Counties,US,"{'rings': [[[-122.12562699946037, 39.414593000..."
6,33105,37969,34909,33084,35970,37115,37950,37532,39225,36225,...,12773,8132,8525,7,06013,US.Counties,Contra Costa County,Query:US.Counties,US,"{'rings': [[[-121.5922400002879, 38.0951800010..."
7,775,714,721,736,722,675,654,599,730,731,...,340,225,198,8,06015,US.Counties,Del Norte County,Query:US.Counties,US,"{'rings': [[[-123.51791100062034, 42.000756000..."
8,4511,5911,5508,4830,4933,4885,5121,5477,6425,5225,...,2897,1706,1498,9,06017,US.Counties,El Dorado County,Query:US.Counties,US,"{'rings': [[[-120.12602699996667, 39.067455000..."
9,39745,36230,35078,38367,40700,36516,31418,27665,26799,37791,...,8555,5495,5683,10,06019,US.Counties,Fresno County,Query:US.Counties,US,"{'rings': [[[-119.00146500008938, 37.570896999..."


In [42]:
m = gis.map('California')
m

In [ ]:
lyr = gis.content.import_data(df=counties_df, title="CA county population")

In [39]:
m.add_layer(lyr.layers[0], {'renderer': 'ClassedColorRenderer', 
                            'field_name':'FEM0'})

### Example: Using comparison levels


The information for the study areas can also be compared with standard geography areas in other levels.
In the example below, the 92373 zip code intersects with both Riverside and San Bernardino counties in California, and hence both these counties as well as the state of California are returned along with the results for the named zip code,  as the comparison levels includes both States and Counties.

In [54]:
enrich(study_areas=[redlands], data_collections=['Age'], 
       comparison_levels=['US.Counties', 'US.States'])

c:\xc\github\geosaurus\src\arcgis\features\_data\geodataset\geodataframe.py:1038: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  frame.set_value(index=idx, col=self._geometry_column_name, value=g)


,FEM0,FEM10,FEM15,FEM20,FEM25,FEM30,FEM35,FEM40,FEM45,FEM5,...,MALE75,MALE80,MALE85,OBJECTID,StdGeographyID,StdGeographyLevel,StdGeographyName,aggregationMethod,sourceCountry,SHAPE
0,871,931,962,1123,1147,1206,1126,1041,1098,891,...,477,317,362,1,92373,US.ZIP5,Redlands,Query:US.ZIP5,US,"{'rings': [[[-117.21461999975689, 34.065140000..."
1,84409,83941,82177,84307,91156,83313,76213,71910,71553,84041,...,26944,18151,16402,2,06065,US.Counties,Riverside County,Query:US.Counties,US,"{'spatialReference': {'wkid': 4326, 'latestWki..."
2,79477,76497,74857,82633,90066,79719,70374,65761,65744,77343,...,17677,10838,9276,3,06071,US.Counties,San Bernardino County,Query:US.Counties,US,"{'spatialReference': {'wkid': 4326, 'latestWki..."
3,1245148,1260883,1269825,1405639,1510193,1429495,1313060,1237282,1250328,1251583,...,408593,268305,266546,4,06,US.States,California,Query:US.States,US,"{'spatialReference': {'wkid': 4326, 'latestWki..."


### Example: Enriching street address

The example below uses a street address as a study area instead.

In [50]:
enrich(study_areas=["380 New York St Redlands CA 92373"], 
       data_collections=['Age'], comparison_levels=['US.Counties', 'US.States'])

c:\xc\github\geosaurus\src\arcgis\features\_data\geodataset\geodataframe.py:1038: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  frame.set_value(index=idx, col=self._geometry_column_name, value=g)


,FEM0,FEM10,FEM15,FEM20,FEM25,FEM30,FEM35,FEM40,FEM45,FEM5,...,StdGeographyName,X,Y,aggregationMethod,areaType,bufferRadii,bufferUnits,bufferUnitsAlias,sourceCountry,SHAPE
0,435,377,417,602,615,577,508,403,386,391,...,NaN,-117.19567,34.056488,BlockApportionment:US.BlockGroups,RingBuffer,1.0,esriMiles,Miles,US,"{'rings': [[[-117.1956703176181, 34.0709967371..."
1,79477,76497,74857,82633,90066,79719,70374,65761,65744,77343,...,San Bernardino County,-117.19567,34.056488,Query:US.Counties,NaN,NaN,NaN,NaN,US,"{'spatialReference': {'wkid': 4326, 'latestWki..."
2,1245148,1260883,1269825,1405639,1510193,1429495,1313060,1237282,1250328,1251583,...,California,-117.19567,34.056488,Query:US.States,NaN,NaN,NaN,NaN,US,"{'spatialReference': {'wkid': 4326, 'latestWki..."


### Example: Buffering locations using non overlapping disks 

The example below creates non-overlapping disks of radii 1, 3 and 5 Miles respectively from a street address and enriches these using the 'Age' data collection.

In [43]:
buffered = BufferStudyArea(area='380 New York St Redlands CA 92373',
                           radii=[1,3,5], units='Miles', overlap=False)
enrich(study_areas=[buffered], data_collections=['Age'])

,FEM0,FEM10,FEM15,FEM20,FEM25,FEM30,FEM35,FEM40,FEM45,FEM5,...,OBJECTID,X,Y,aggregationMethod,areaType,bufferRadii,bufferUnits,bufferUnitsAlias,sourceCountry,SHAPE
0,435,377,417,602,615,577,508,403,386,391,...,1,-117.19567,34.056488,BlockApportionment:US.BlockGroups,RingBufferBands,1,Miles,Miles,US,"{'rings': [[[-117.1956703176181, 34.0709967371..."
1,1759,1809,2158,2280,2404,2369,2060,1908,1941,1786,...,2,-117.19567,34.056488,BlockApportionment:US.BlockGroups,RingBufferBands,3,Miles,Miles,US,"{'rings': [[[-117.1956703176181, 34.1000138698..."
2,2790,2776,2628,3257,3596,2975,2538,2364,2453,2807,...,3,-117.19567,34.056488,BlockApportionment:US.BlockGroups,RingBufferBands,5,Miles,Miles,US,"{'rings': [[[-117.1956703176181, 34.1290308652..."


### Example: Using drive times as study areas
    
The example below creates 5 and 10 minute drive times from a street address and enriches these using the 'Age' data collection.

In [44]:
buffered = BufferStudyArea(area='380 New York St Redlands CA 92373', 
                           radii=[5, 10], units='Minutes', 
                           travel_mode='Driving')
drive_time_df = enrich(study_areas=[buffered], data_collections=['Age'])

In [45]:
drive_time_df

,FEM0,FEM10,FEM15,FEM20,FEM25,FEM30,FEM35,FEM40,FEM45,FEM5,...,OBJECTID,X,Y,aggregationMethod,areaType,bufferRadii,bufferUnits,bufferUnitsAlias,sourceCountry,SHAPE
0,367,319,345,484,509,463,398,320,305,329,...,1,-117.19567,34.056488,BlockApportionment:US.BlockGroups,NetworkServiceArea,5,Minutes,Drive Time Minutes,US,"{'rings': [[[-117.1989212044467, 34.0751628879..."
1,2299,2256,2651,2988,3075,2945,2577,2317,2329,2263,...,2,-117.19567,34.056488,BlockApportionment:US.BlockGroups,NetworkServiceArea,10,Minutes,Drive Time Minutes,US,"{'rings': [[[-117.20049285913858, 34.130182266..."


### Visualize results on a map

The returned spatial dataframe can be visualized on a map as shown below:

In [48]:
redlands_map = gis.map('Redlands, CA')
redlands_map.basemap = 'dark-gray-vector'
redlands_map

![](http://esri.github.io/arcgis-python-api/notebooks/nbimages/guide_ge_drive_times_03.png)

In [49]:
redlands_map.draw(drive_time_df.to_featureset())

## Saving GeoEnrichment Results

The results can be saved back to a GIS as a feature layer 

#### Example: Save Data to a Feature Layer

In [48]:
gis.content.import_data(df=drive_time_df, title="Age statistics within 5,10 minutes of drive time from Esri")

<Item title:"Age statistics within 5,10 minutes of drive time from Esri" type:Feature Layer Collection owner:singhqa>